In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\Vador66\\Project\\Chest_Cancer'

In [19]:
#entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path : Path
    updated_base_model_path: Path
    training_data : Path
    params_is_augmentation : bool
    params_image_size : list
    params_batch_size : int
    params_epochs : int

In [20]:
from Chest_Cancer_Classifier.constants import *
from Chest_Cancer_Classifier.utils.common import read_yaml, create_directories

In [26]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [14]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
#components
class Training0:
    def __init__(self, config:TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)


    #to split data in train and valid 
    def train_valid_generator(self):
        # Arguments pour les Générateurs de Données
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0 # for valid set
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        #Création du Générateur de Données pour la Validation pas besoin
        '''valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )'''
        #Générateur de Données pour l'Entraînement avec ou sans Augmentation
        if self.config.params_is_augmentation :
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
                )
        else : 
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        #else:
        #   train_datagenerator = valid_datagenerator

        #Flux de Données pour l'Entraînement
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            shuffle=True,
            **dataflow_kwargs
        )


In [15]:
 #from keras doc
import os
import numpy as np
import keras
from keras import layers
from tensorflow import data as tf_data
import matplotlib.pyplot as plt

In [27]:
test = ConfigurationManager()
training_model = test.get_training_config()

[2024-08-28 15:57:49,341: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-28 15:57:49,343: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-28 15:57:49,344: INFO: common: created directory at: artifacts]
[2024-08-28 15:57:49,345: INFO: common: created directory at: artifacts\training]


In [28]:
training_model.training_data

WindowsPath('artifacts/data_ingestion/Data')

In [39]:
# Chemin vers le répertoire de données d'entraînement
training_data_path = training_model.training_data

# Liste pour stocker les noms des dossiers
folders = []

# Parcourir les éléments dans le répertoire
for item in os.listdir(training_data_path):
    item_path = os.path.join(training_data_path, item)
    if os.path.isdir(item_path):
        folders.append(item)

# Afficher les dossiers
print("Dossiers dans training_model.training_data:")
for folder in folders:
    print(folder)

Dossiers dans training_model.training_data:
test
train
valid


In [31]:
folders

['test', 'train', 'valid']

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image_dataset_from_directory

In [56]:
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        validation_split = 0.2,
                                  
        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255)

In [57]:
train_dataset  = train_datagen.flow_from_directory(
                    directory = os.path.join(training_model.training_data,'train'),
                    target_size = (224,224),
                    class_mode = 'categorical',
                    batch_size = 64
                    )

Found 613 images belonging to 4 classes.


In [58]:
val_dataset  = valid_datagen.flow_from_directory(
                    directory = os.path.join(training_model.training_data,'valid'),
                    target_size = (224,224),
                    class_mode = 'categorical',
                    batch_size = 64
                    )

Found 72 images belonging to 4 classes.


In [59]:
test_dataset  = test_datagen.flow_from_directory(
                    directory = os.path.join(training_model.training_data,'test'),
                    target_size = (224,224),
                    class_mode = 'categorical',
                    batch_size = 64
                    )

Found 315 images belonging to 4 classes.


In [61]:
class Training:
    def __init__(self, config:TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):

        train_datagen = ImageDataGenerator(
            rescale = 1./255,
            validation_split = 0.2,
                                    
            rotation_range=5,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            #zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='nearest')

        valid_datagen = ImageDataGenerator(rescale = 1./255,
                                        validation_split = 0.2)

        test_datagen  = ImageDataGenerator(rescale = 1./255)

        self.train_dataset  = train_datagen.flow_from_directory(
                    directory = os.path.join(self.config.training_data,'train'),
                    target_size = (224,224),
                    class_mode = 'categorical',
                    batch_size = self.config.params_batch_size
                    )
        self.val_dataset  = valid_datagen.flow_from_directory(
                    directory = os.path.join(self.config.training_data,'valid'),
                    target_size = (224,224),
                    class_mode = 'categorical',
                    batch_size = self.config.params_batch_size
                    )
        self.test_dataset  = test_datagen.flow_from_directory(
                    directory = os.path.join(self.config.training_data,'test'),
                    target_size = (224,224),
                    class_mode = 'categorical',
                    batch_size = self.config.params_batch_size
                    )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        self.steps_per_epoch = self.train_dataset.samples // self.train_dataset.batch_size
        self.validation_steps = self.val_dataset.samples // self.val_dataset.batch_size

        self.model.fit(
            self.train_dataset,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.val_dataset
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [64]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-08-28 17:02:45,819: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-28 17:02:45,822: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-28 17:02:45,823: INFO: common: created directory at: artifacts]
[2024-08-28 17:02:45,824: INFO: common: created directory at: artifacts\training]
Found 613 images belonging to 4 classes.
Found 72 images belonging to 4 classes.
Found 315 images belonging to 4 classes.
Epoch 1/2
38/38 [==============================] - 77s 2s/step - loss: 22.6813 - accuracy: 0.2948 - val_loss: 24.0968 - val_accuracy: 0.3125
Epoch 2/2
38/38 [==============================] - 79s 2s/step - loss: 23.0950 - accuracy: 0.2630 - val_loss: 13.6723 - val_accuracy: 0.3750
